In [2]:
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip

# Tạo SparkSession có hỗ trợ Delta Lake
builder = SparkSession.builder \
    .appName("QueryDeltaHistory") \
    .master("local[*]") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()


25/05/24 15:13:25 WARN Utils: Your hostname, LAPTOP-PMSG8QVF resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/05/24 15:13:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/pc/.ivy2/cache
The jars for the packages stored in: /home/pc/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2647d065-8bb2-4aa6-a2fe-13456c80364a;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.3.1 in central
	found io.delta#delta-storage;3.3.1 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 171ms :: artifacts dl 7ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.3.1 from central in [default]
	io.delta#delta-storage;3.3.1 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0 

In [3]:
df_v1 = spark.read.format("delta") \
    .option("versionAsOf", 1) \
    .load("hdfs://localhost:9000/user/minhhieu/delta/gold/weather_features")

df_v1.createOrReplaceTempView("weather_v1")

spark.sql("""
    SELECT year, month, COUNT(*) AS rainy_days
    FROM weather_v1
    WHERE LOWER(weather) LIKE '%rain%'
    GROUP BY year, month
    ORDER BY rainy_days DESC
""").show()


25/05/24 15:13:33 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
[Stage 3:>                                                          (0 + 1) / 1]

+----+-----+----------+
|year|month|rainy_days|
+----+-----+----------+
|2019|    9|       160|
|2019|    8|       147|
|2019|    7|       140|
|2019|   10|       137|
|2019|    6|       126|
|2019|    5|       106|
|2019|   11|        98|
|2018|    9|        67|
|2017|   10|        66|
|2018|    8|        65|
|2018|    6|        63|
|2018|    7|        60|
|2017|    5|        59|
|2017|    8|        52|
|2017|    9|        52|
|2017|    6|        50|
|2017|    7|        47|
|2018|   10|        44|
|2018|    5|        43|
|2019|    4|        38|
+----+-----+----------+
only showing top 20 rows



In [4]:
# Đọc version 0
df_v0 = spark.read.format("delta") \
    .option("versionAsOf", 0) \
    .load("hdfs://localhost:9000/user/minhhieu/delta/gold/weather_features")

# Tạo view tạm để dùng SQL
df_v0.createOrReplaceTempView("weather_v0")

# Truy vấn
spark.sql("""
    SELECT year, month, COUNT(*) AS hot_days
    FROM weather_v0
    WHERE temperature > 35
    GROUP BY year, month
    ORDER BY year, month
""").show()


+----+-----+--------+
|year|month|hot_days|
+----+-----+--------+
|2017|    3|       4|
|2017|    4|      16|
|2018|    2|       4|
|2018|    3|      15|
|2018|    4|      17|
|2018|    5|       6|
|2019|    1|       3|
|2019|    2|       6|
|2019|    3|      35|
|2019|    4|      39|
|2019|    5|      20|
|2019|    6|       4|
+----+-----+--------+

